<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500,>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<h2 style="text-align: center;"><b>Задание</b></h2>

Добейтесь как можно лучшего качества в соревновании!  

[DLSchool: clothes recognition](https://www.kaggle.com/c/dlschool-fashionmnist3)

*Рекомендация*: попробуйте поменять количество итераций, количество нейронов, количество слоёв, гиперпараметры сети (learning_rate, метод оптимизации вместо SGD можно взять другой)

In [1]:
import pandas as pd
import numpy as np
import torch

In [11]:
# Загружаем данные
TRAIN_PATH = './fashionmnist/fashion-mnist_train.csv'
TEST_PATH = './fashionmnist/fashion-mnist_test.csv'

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

X_train = train_df.values[:, 1:]
y_train = train_df.values[:, 0]

X_test = test_df.values # [:, 1:]  # удаляем столбец 'label'



X_train_tensor = torch.cuda.FloatTensor(X_train)
y_train_tensor = torch.cuda.LongTensor(y_train.astype(np.int64))

X_test_tensor = torch.cuda.FloatTensor(X_test)


length = y_train_tensor.shape[0]
num_classes = 10  # количество классов, в нашем случае 10 типов одежды

# закодированные OneHot-ом метки классов
y_onehot = torch.cuda.FloatTensor(length, num_classes)

y_onehot.zero_()
y_onehot.scatter_(1, y_train_tensor.view(-1, 1), 1)


tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]], device='cuda:0')

In [12]:
def generate_batches(X, y, batch_size=64):
    for i in range(0, X.shape[0], batch_size):
        X_batch, y_batch = X[i:i+batch_size], y[i:i+batch_size]
        yield X_batch, y_batch

In [5]:
def accuracy(predict):
#     Выбираем максимальное значение по 1 оси
    _, predict = torch.max(predict, 1)

#     Сравниваем предсказания с тестовой выборкой
    c = (predict == y_train_tensor.cpu()[:10000])
    print("Accuracy:", round(c.sum().item() / len(c) * 100, 2), "\n")

def write_csv(predict):
    _, predict = torch.max(predict.cpu(), 1)
    answer_df = pd.DataFrame(data=predict.data.numpy(), columns=['Category'])
    answer_df['Id'] = answer_df.index
    answer_df.to_csv('./baseline_gart.csv', index=False)


In [6]:
def train(model, batch=64, epochs=100, learning_rate= 1e-4):
    print(batch, epochs, learning_rate)

#     Функция потерь
    loss_fn = torch.nn.CrossEntropyLoss().cuda()

#     Оптимизатор
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.cuda()

    for epoch_num  in range(epochs):
#         Батчи
        for X_batch, y_batch in generate_batches(X_train_tensor, 
                                                 y_train_tensor, 
                                                 batch):
            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch)
        
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
    return model


In [14]:
X_train_tensor = torch.cuda.FloatTensor(X_train.reshape(-1,1,28,28))
y_train_tensor = torch.cuda.LongTensor(y_train.astype(np.int64))

X_test_tensor = torch.FloatTensor(X_test.reshape(-1,1,28,28))

class Flatten(torch.nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    

In [15]:
def test_write(my_model):
    my_model = my_model.cpu()
    predict = my_model(X_train_tensor.cpu()[:10000])
    accuracy(predict)

    write_csv(my_model(X_test_tensor))
#    Чистим мусор за собой
    my_model = None
    predict = None
    torch.cuda.empty_cache()

In [22]:
def get_model(id_model):
    if id_model == 1:
        return torch.nn.Sequential(torch.nn.Conv2d(1, 32, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(32).cuda(),

                                   torch.nn.Conv2d(32, 32, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(32).cuda(),

                                   torch.nn.MaxPool2d(2).cuda(),
                                   torch.nn.Dropout(p=0.2).cuda(),

                                   Flatten().cuda(),

                                   torch.nn.Linear(4608, 32).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.Linear(32, 10).cuda(),
                                  ).cuda()
    elif id_model == 2:
        return torch.nn.Sequential(torch.nn.Conv2d(1, 32, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(32).cuda(),

                                   torch.nn.Conv2d(32, 32, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(32).cuda(),

                                   torch.nn.MaxPool2d(2).cuda(),
                                   torch.nn.Dropout(p=0.2).cuda(),

                                   torch.nn.Conv2d(32, 64, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(64).cuda(),

                                   torch.nn.Conv2d(64, 64, 3).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.BatchNorm2d(64).cuda(),

                                   torch.nn.MaxPool2d(2).cuda(),
                                   torch.nn.Dropout(p=0.2).cuda(),

                                   Flatten().cuda(),

                                   torch.nn.Linear(1024, 32).cuda(),
                                   torch.nn.ReLU().cuda(),
                                   torch.nn.Linear(32, 10).cuda(),
                                  ).cuda()


In [23]:
# Быстрое обучение, премлемая точность
my_model = train(get_model(1), 
                 batch=100,
                 learning_rate= 1e-3,
                 epochs=7)

test_write(my_model)

100 7 0.001
Accuracy: 96.35 



In [ ]:
my_model = train(get_model(2), 
                 batch=100,
                 learning_rate= 1e-3,
                 epochs=17)

test_write(my_model)

100 17 0.001
